In [43]:
## import all necessary libraries ##

import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import copy

#########################

from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.filtering.log.attributes import attributes_filter
from pm4py.algo.filtering.log.variants import variants_filter
from pm4py.statistics.traces.log import case_statistics
from pm4py.objects.log.util import insert_classifier
from pm4py.util import constants

#########################

import distance

from similarity.levenshtein import Levenshtein
levenshtein = Levenshtein()

from similarity.damerau import Damerau
damerau = Damerau()

from pyjarowinkler import distance as jwdistance
from similarity.jarowinkler import JaroWinkler
jarowinkler = JaroWinkler()

from similarity.weighted_levenshtein import WeightedLevenshtein
from similarity.weighted_levenshtein import CharacterSubstitutionInterface
import math
from random import sample
from random import seed
class CharacterSubstitution(CharacterSubstitutionInterface):
    def cost(self, c0, c1):
        return math.inf # assign inifte weight to all substitutions
levenshtein2 = WeightedLevenshtein(CharacterSubstitution())

#########################

import pomegranate as pom
from sklearn import model_selection as ms
import random 

# LOF

## Distance class

In [71]:
class Distance:
    
    ### SETUP ###
    
    ## load dataset, generate mapping and generate strings
    def __init__(self, log):
        self.log = log
        
        self.strings = []
        self.matrix = []
        self.transl = {}
        self.variant_to_Vindex = {}
        self.index_to_variant = [] 
        #seed(1633048)
        #self.log = sample(self.log, int(len(self.log)/10))
        self.clear_caches()
        
        self.gen_trace_to_Tindex()
        
        self.gen_mapping()
        self.gen_variant_strings()        
    
    def clear_caches(self):
        self.Nk_res_dict = {} # N_k result cache
    
    ## generate mapping from activity to char
    def gen_mapping(self):
        activities = list(attributes_filter.get_attribute_values(self.log, "customClassifier").keys())
        for i, a in enumerate(activities):
            self.transl[a] = chr(i+1)

    def gen_trace_to_Tindex(self):
        self.trace_to_Tindex = {}
        for i, t in enumerate(self.log):
            self.trace_to_Tindex[t] = i
            
            
    ## generate strings for all variants
    def gen_variant_strings(self):
        self.variants = variants_filter.get_variants(self.log, parameters={variants_filter.Parameters.ACTIVITY_KEY: "customClassifier"}) # all variants as dictionary
        variant_strings = list(self.variants.keys()) # variants as strings
        self.variant_to_index = {} # dictionary to translate variant to index in list for later lookup of traces
        
        for i, v in enumerate(variant_strings):
            string = ""
            for e in v.split(","):
                string = string + self.transl[e] 
            
            #self.strings.append(list_to_string(v.split(",")))
            self.strings.append(string)
            
            self.variant_to_index[v] = i
            self.index_to_variant.append(v)
            
        print("Number of variants: " + str(len(self.strings)))
    
    ### CALCULATION ###
    
    ## calculate distance matrix
    def calculate(self):
        n = len(self.strings)
        self.matrix = np.full((n, n), 0, dtype = np.uint8)

        for i, x in enumerate(self.strings):
            for j, y in enumerate(self.strings):
                if j >= i: # only calculate upper right triangle of matrix
                    #dist = distance.hamming(x, y)
                    dist = levenshtein.distance(x, y)
                    #dist = levenshtein2.distance(x, y)
                    #dist = damerau.distance(x, y)
                    #dist = (1- jarowinkler.similarity(x, y))*255
                    #print(dist)
                    self.matrix[i][j] = dist

        # mirror upper right triangle of matrix by adding the transposition
        self.matrix = self.matrix + self.matrix.T

        return self.matrix
    
    ### RETRIEVAL ###
    
    ## translate trace to its corresponding matrix index
    def trace_to_index(self, trace):
        # convert trace to string tion of variant (concept:name separated by commas)
        trace_string = ""
        for e in trace:
            #trace_string = trace_string + e["concept:name"] + ","
            trace_string = trace_string + e["customClassifier"] + ","
        trace_string = trace_string[:-1] # remove last comma

        return self.variant_to_index[trace_string]
    
    ## translate matrix (variant) index to trace indices
    def index_to_traces(self, i):
        variant_string = self.index_to_variant[i]
        traces = self.variants[variant_string] # retrieve traces from variant dictionary
        filtered_variants = {variant_string: traces} # generate new variants dictionary with only one variant
        
        filtered_log = variants_filter.apply(self.log, filtered_variants, parameters={variants_filter.Parameters.ACTIVITY_KEY: "customClassifier"})
        traces = []
        for t in filtered_log:
            traces.append(t)
        
        return traces
        
    
    ## retrieve distance of two traces from matrix
    def dist(self, t1, t2):
        i1 = self.trace_to_index(t1)
        i2 = self.trace_to_index(t2)
        return self.matrix[i1][i2]
    
    # return traces of k nearest neighbors of A
    def N_k(self, k, A):
        A_variant_index = self.trace_to_index(A)
        if A_variant_index in self.Nk_res_dict.keys(): # check result cache
            #print("N_k cache hit")
            return self.Nk_res_dict[A_variant_index]
        else:
            idx_sort = np.argsort(self.matrix[A_variant_index]) # indices of neighbors in ascending distance

            i = 0
            N_k_traces = []
            
            while len(N_k_traces) < k or self.matrix[A_variant_index][idx_sort[i-1]] == self.matrix[A_variant_index][idx_sort[i]]:
                N_k_traces.extend(self.index_to_traces(idx_sort[i]))
                if A in N_k_traces:
                    N_k_traces.remove(A)
                i = i + 1
                if i == len(self.matrix): # if all traces have been added
                    break
                
            
            self.Nk_res_dict[A_variant_index] = N_k_traces
            return N_k_traces

## LOF class

In [72]:
class LOF:
    
    def __init__(self, log):
        self.dist = Distance(log)
        self.dist.calculate()
        self.clear_caches()
    
    def clear_caches(self):
        self.dist.clear_caches()
        
        # result caches
        self.kd_res_dict = {}
        self.rd_res_dict = {}
        self.lof_res_dict = {}
        self.lrd_res_dict = {}
    
    
    ## k-distance
    def k_distance(self, k, A):
        A_variant_index = self.dist.trace_to_index(A)
        if A_variant_index in self.kd_res_dict.keys(): # check result cache
            #print("k_distance cache hit")
            return self.kd_res_dict[A_variant_index]
        else:
            N_k = self.dist.N_k(k, A)
            k_variant_index = self.dist.trace_to_index(N_k[-1])
            A_variant_index = self.dist.trace_to_index(A)

            res = self.dist.matrix[A_variant_index][k_variant_index] # retrieve distance from k-th nearest neighbor (-1 to offset arraystart, +1 to not include trace itself)
            self.kd_res_dict[A_variant_index] = res
            return res
        
    ## reachability distance
    def reachability_dist(self, k, A, B):
        A_variant_index = self.dist.trace_to_index(A)
        B_variant_index = self.dist.trace_to_index(B)
        if (A_variant_index, B_variant_index) in self.rd_res_dict.keys(): # check result cache
            #print("rd cache hit")
            return self.rd_res_dict[(A_variant_index, B_variant_index)]
        else:
            res = max(self.k_distance(k, B), self.dist.matrix[A_variant_index][B_variant_index])
            self.rd_res_dict[(A_variant_index, B_variant_index)] = res
            return res
    
    ## local reachability density
    def lrd(self, k, A):
        A_variant_index = self.dist.trace_to_index(A)
        if A_variant_index in self.lrd_res_dict.keys(): # check result cache
            #print("lrd cache hit")
            return self.lrd_res_dict[A_variant_index]
        else:
            
            N_k = self.dist.N_k(k, A)            
            sum = 0
            for b in N_k:
                   sum = sum + self.reachability_dist(k, A, b) # sum of rachability distances in k-Neighborhood
            
            res = 1 / (sum / len(N_k))
            self.lrd_res_dict[A_variant_index] = res
            return res
        
    def lof(self, k, A):
        A_variant_index = self.dist.trace_to_index(A)
        if A_variant_index in self.lof_res_dict.keys(): # check result cache
            #print("lof cache hit")
            return self.lof_res_dict[A_variant_index]
        else:
            
            N_k = self.dist.N_k(k, A)
            sum = 0
            for b in N_k:
                sum = sum + self.lrd(k, b)

            res = sum / (len(N_k) * self.lrd(k, A))
            self.lof_res_dict[A_variant_index] = res
            return res
    
    def calculate(self, k):
        res = np.array([])
        
        for a in self.dist.log:
            res = np.append(res, self.lof(k, a))
            
        return res
    

# Run

## Import log

In [386]:
path = "Datasets/BPIC20_sample.xes"
log = xes_importer.apply(path)

# generate custom activity classifier
try:
    
    #log, activity_key = insert_classifier.insert_activity_classifier_attribute(self.log, "Activity classifier")
    for trace in log:
        for event in trace:
            custom_classifier = ""
            #loop through all attributes which classify the activity
            for activity_classifier in log.classifiers["Activity classifier"]:
                #generate new attribute combining all classifying attributes
                custom_classifier = custom_classifier + event[activity_classifier] + "+"
            custom_classifier = custom_classifier[:-1]
            event["customClassifier"] = custom_classifier
except: 
    print("foo")
    for trace in log:
        for event in trace:
            event["customClassifier"] = event["concept:name"]

# generate index attribute for each event (later used to fiter)
for trace in log:
    for i, event in enumerate(trace):
        event["index"] = i


parsing log, completed traces ::   0%|          | 0/706 [00:00<?, ?it/s]

foo


## Generate LOFs

In [347]:
start = datetime.now()
print(start)

# find length of longest trace
trace_len = [len(i) for i in log]
max_trace_len = max(trace_len)
print("max trace length: " + str(max_trace_len))

res = []

for l in range(0, max_trace_len): # iterate from 1 to length of longest trace
    print("l: " + str(l+1))
    # filter events by attribute "index". only events with index between 0 and l are kept
    log_tmp = attributes_filter.apply_numeric_events(log, 0, l, 
                                                     parameters={constants.PARAMETER_CONSTANT_ATTRIBUTE_KEY: "index"})
    
    # run LOF calculation on filtered log
    lof = LOF(log_tmp)

    k = len(max(lof.dist.variants.values(), key=len)) + 1 # no. traces in largest variant + 1
    res_tmp = lof.calculate(k)
    res.append(res_tmp)
    
end = datetime.now()
print(end-start)

2022-01-03 15:32:38.792277
max trace length: 104
l: 1
Number of variants: 1
l: 2
Number of variants: 3
l: 3
Number of variants: 3
l: 4
Number of variants: 3


<ipython-input-72-6213939bfada>:58: RuntimeWarning: divide by zero encountered in double_scalars
  res = 1 / (sum / len(N_k))
<ipython-input-72-6213939bfada>:74: RuntimeWarning: invalid value encountered in double_scalars
  res = sum / (len(N_k) * self.lrd(k, A))


l: 5
Number of variants: 6
l: 6
Number of variants: 12
l: 7
Number of variants: 19
l: 8
Number of variants: 26
l: 9
Number of variants: 31
l: 10
Number of variants: 41
l: 11
Number of variants: 48


KeyboardInterrupt: 

In [298]:
# mirror results and rotate by 270 degrees
res_rot = np.rot90(res[::-1], 3) 

In [299]:
# write to csv
np.savetxt("res_BPIC17_sample.csv", res_rot, delimiter=",")

In [387]:
# read from csv
res_rot = np.genfromtxt("res_BPIC20_sample.csv", delimiter=",")
res = np.rot90(res_rot)
res = res[::-1]

In [331]:
res_rot

array([[1.        , 0.99998517, 1.81649132, ..., 5.14166176, 5.14166176,
        5.14166176],
       [1.        , 0.99998517, 1.78464626, ..., 1.63485424, 1.63485424,
        1.63485424],
       [1.        , 0.99998517, 1.8435242 , ..., 3.34324685, 3.34324685,
        3.34324685],
       ...,
       [1.        , 0.99998517, 0.99211203, ..., 0.99366459, 0.99366459,
        0.99366459],
       [1.        , 0.99998517, 0.99211203, ..., 0.99366459, 0.99366459,
        0.99366459],
       [1.        , 0.99998517, 1.04075619, ..., 1.08762159, 1.08762159,
        1.08762159]])

In [388]:
# define cutoffs for anomaly at each length
cutoffs95 = []
cutoffs85 = []
cutoffs75 = []
cutoffs50 = []
for l in range(len(res)):
    r = []
    for t in range(len(res_rot)):
        if len(log[t]) >= l + 1:
            r.append(res[l][t])
    cutoffs50.append(np.percentile(r, 50))
    cutoffs85.append(np.percentile(r, 85))
    cutoffs95.append(np.percentile(r, 95))

In [389]:
# classify traces at each point
classification = [None] * len(res_rot)

for j, trace_res in enumerate(res_rot):
    classification_trace = [None] * len(log[j])
    for i, r in enumerate(trace_res):
        if i < len(log[j]):
            if r > cutoffs50[i]:
                classification_trace[i] = "anomaly"
            else:
                classification_trace[i] = "no anomaly"
    classification[j] = classification_trace

In [167]:
cutoffs75

[1.0,
 0.9960172390636931,
 1.0141779837752392,
 1.449660175242035,
 1.483283527196964,
 1.5968136913981197,
 1.743271442513729,
 1.598951061104271,
 1.8314951912122028,
 2.100894225300608,
 2.3367419705205403,
 2.6120536165973642,
 2.8872118451840745,
 3.1726639320901615,
 3.4241994237131523,
 3.7756192754700315,
 4.0758992434821435,
 4.310665017564494,
 4.682986065340565,
 5.039655983205424,
 5.3576448784553605,
 5.634344693952617,
 6.159672640524114,
 6.491456792186284,
 6.770600294497551,
 7.0859214470283,
 7.410911005504505,
 7.553470223855971,
 7.858152670901335,
 8.578045471432864,
 8.913729983303785,
 9.22343583420026,
 9.558610919115036,
 9.877995045915053,
 9.873410580392505]

In [390]:
na = 0
a = 0
for t in classification:
    for c in t:
        if c == "no anomaly":
            na = na+1
        elif c == "anomaly":
            a = a+1
            
t = na + a
print(na/t)
print(a/t)

0.6004982448193863
0.39950175518061376


## Setup data

In [394]:
lists = []
for trace in log:
    tlist = []
    for i, event in enumerate(trace):
        tlist.append(event["customClassifier"])
    lists.append(tlist)

#random.seed(1633048)

# split data into training and test
lists_train, lists_test, is_anomaly_train, is_anomaly_test = ms.train_test_split(lists, classification, test_size = 0.2, random_state = 1)

In [336]:
sum(map(len, lists_train))/float(len(lists_train))

12.50531914893617

## Setup model
- Discrete observations
- Two states

In [395]:
model1 = pom.HiddenMarkovModel.from_samples(
    pom.DiscreteDistribution, 
    n_components=2,
    X=lists_train, 
    labels=is_anomaly_train,
    #state_names=["anomaly", "no anomaly", "unknown"],
    state_names=["no anomaly", "anomaly"],
    algorithm="labeled")
model1.bake()

In [290]:
model1.states

[{
     "class" : "State",
     "distribution" : {
         "class" : "Distribution",
         "dtype" : "numpy.str_",
         "name" : "DiscreteDistribution",
         "parameters" : [
             {
                 "01_BB_540" : 0.0010136847440446021,
                 "01_BB_630" : 0.0,
                 "01_BB_770" : 0.0015205271160669033,
                 "01_BB_775" : 0.0010136847440446021,
                 "01_HOOFD_010" : 0.00912316269640142,
                 "01_HOOFD_011" : 0.006588950836289914,
                 "01_HOOFD_015" : 0.021794221996958945,
                 "01_HOOFD_020" : 0.015712113532691332,
                 "01_HOOFD_030_1" : 0.02331474911302585,
                 "01_HOOFD_030_2" : 0.02280790674100355,
                 "01_HOOFD_040" : 0.013177901672579827,
                 "01_HOOFD_050" : 0.00963000506842372,
                 "01_HOOFD_055" : 0.00506842372022301,
                 "01_HOOFD_060" : 0.008109477952356817,
                 "01_HOOFD_061" : 0.01267

In [338]:
model1.dense_transition_matrix()

array([[0.74709884, 0.25290116, 0.        , 0.        ],
       [0.22681704, 0.77318296, 0.        , 0.        ],
       [0.5       , 0.5       , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]])

## Evaluate

In [326]:
false_positive = 0 # anomaly incorrectly detected
false_negative = 0 # anomaly incrrectly not detected
true_positive = 0 # anomaly correctly detectd
true_negative = 0 # anomaly correclty not detected

for i, t in enumerate(lists_test):
    prediction = model1.predict(t)
    if prediction[-1] == 0 and is_anomaly_test[i][-1] == "no anomaly":
        true_negative = true_negative + 1
    if prediction[-1] == 0 and is_anomaly_test[i][-1] == "anomaly":
        false_negative = false_negative + 1
    if prediction[-1] == 1 and is_anomaly_test[i][-1] == "no anomaly":
        false_positive = false_positive + 1
    if prediction[-1] == 1 and is_anomaly_test[i][-1] == "anomaly":
        true_positive = true_positive + 1

TPR = true_positive / (true_positive + false_negative)
TNR = true_negative / (true_negative + false_positive)
        
FPR = false_positive / (false_positive + true_negative)
FNR = false_negative / (false_negative + true_positive)

print("FPR\t\t\t" + str(round(FPR * 100, 1)) + "%")
print("FNR\t\t\t" + str(round(FNR * 100, 1)) + "%")
print("\n")
print("TNR (specificity)\t" + str(round(TNR * 100, 1)) + "%")
print("TPR (sensitivity)\t" + str(round(TPR * 100, 1)) + "%")

FPR			100.0%
FNR			0.0%


TNR (specificity)	0.0%
TPR (sensitivity)	100.0%


In [207]:
print(model1.predict(lists_test[0]))
print(is_anomaly_test[0])

[1, 1, 1, 1, 1]
['no anomaly', 'no anomaly', 'anomaly', 'anomaly', 'no anomaly']


In [208]:
print("neg: " + str(false_positive + true_negative))
print("pos: " + str(false_negative + true_positive))


neg: 281
pos: 17


In [396]:
## rates over all events
tp = 0
tn = 0
fp = 0
fn = 0

for i, t in enumerate(lists_test):

    prediction = model1.predict(t)
    for j, p in enumerate(prediction):
        p_name = model.states[p].name
        if p_name == "no anomaly" and is_anomaly_test[i][j] == "no anomaly":
            tn = tn + 1
        elif p_name == "anomaly" and is_anomaly_test[i][j] == "anomaly":
            tp = tp + 1
        elif p_name == "no anomaly" and is_anomaly_test[i][j] == "anomaly":
            fn = fn + 1
        elif p_name == "anomaly" and is_anomaly_test[i][j] == "no anomaly":
            fp = fp + 1

print("TPR (sensitivity)\t" + str(round(tp / (tp+fn) * 100, 1)))
print("TNR (specificity)\t" + str(round(tn / (tn + fp) * 100, 1)))
print("Error Rate\t\t" + str(round((fp + fn)/(tp+tn+fp+fn) * 100, 1)))



TPR (sensitivity)	69.9
TNR (specificity)	61.2
Error Rate		34.9


In [397]:
# rates weighted over traces
tp_list = []
tn_list = []
err_list = []

for i, t in enumerate(lists_test):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    err = 0
    
    prediction = model1.predict(t)
    for j, p in enumerate(prediction):
        if p == 1 and is_anomaly_test[i][j] == "no anomaly":
            tn = tn + 1
        elif p == 0 and is_anomaly_test[i][j] == "anomaly":
            tp = tp + 1
        elif p == 1 and is_anomaly_test[i][j] == "anomaly":
            fn = fn + 1
        elif p == 0 and is_anomaly_test[i][j] == "no anomaly":
            fp = fp + 1
            
        if (tp+fn) != 0: tp_list.append(tp / (tp + fn))
        if (tn+fp) != 0: tn_list.append(tn / (tn + fp))
        err_list.append((fp + fn) / (tp+tn+fp+fn))


print("TPR (sensitivity)\t" + str(round(sum(tp_list) / len(tp_list) * 100, 1)))
print("TNR (specificity)\t" + str(round(sum(tn_list) / len(tn_list) * 100, 1)))
print("Error Rate\t\t" + str(round(sum(err_list) / len(err_list) * 100, 1)))



TPR (sensitivity)	63.9
TNR (specificity)	75.8
Error Rate		26.3
